In [1]:
import simpy
import numpy as np
import seaborn as sns
import scipy as sc

### Se está diseñando un web service, el cual cada vez que es invocado consulta a una base de datos.
### Se estima que el tiempo que transcurre entre cada llamada al servicio se puede modelar según una distribución exponencial con media 𝜇 = 4 𝑠𝑒𝑔𝑢𝑛𝑑𝑜𝑠
### Se debe decidir la arquitectura de base de datos a utilizar entre las dos siguientes:
### 1) Utilizar 2 bases de datos distribuidas. Con probabilidad 𝑝 = 0.6 las solicitudes son atendidas por la base 1 y con probabilidad 𝑞 = 1 − 𝑝 son atendidos por la base de datos 2. El tiempo que demora cada base de datos en atender una solicitud sigue una distribución exponencial con medias, 𝜇1 = 0,7 𝑠𝑒𝑔 y 𝜇2 = 1 𝑠𝑒𝑔 respectivamente.
### 2) Utilizar 1 base de datos central. En este caso la demora en resolver una solicitud sigue una distribución exponencial con 𝜇 = 0,8 𝑠𝑒𝑔𝑢𝑛𝑑𝑜𝑠 
### Simular para cada opción 100000 solicitudes procesadas, determinando:
####  - El tiempo medio de espera entre que la solicitud llega y puede ser procesada (suponer que ninguna conexión cae por timeout).
####  - La fracción de las solicitudes que no esperaron para ser procesadas.
####  - La opción 1 es más costosa que la segunda opción y la empresa sólo acepta realizar la inversión si el tiempo medio que demora en resolver cada solicitud (tiempo en fila + tiempo de procesamiento) es como mínimo 50% menor que la opción 2. ¿Qué solución le recomienda?

In [36]:
class web_service_1:
    def __init__(self, env, media_call, bdd_1, bdd_2, proba_b1, media_1, media_2, sol_max):
        self.env = env
        self.media_call = media_call
        self.bdd_1 = bdd_1
        self.bdd_2 = bdd_2
        self.proba_b1 = proba_b1
        self.media_1 = media_1
        self.media_2 = media_2
        self.sol_max = sol_max
        self.action = env.process(self.run(env))
        self.usos_bdd = [0,0]
        self.cant_sol = 0
        self.cant_sol_sin_espera = 0
        self.tiempo_cola = 0
        self.tiempo_espera_tot = 0 #Sumatoria Tiempo_proc_i - Tiempo_llamada_i =
                                   # (Sumatoria Tiempo_proc_i) - (Sumatoria Tiempo_llamada_k)
    
    def run(self, env):
        while(self.cant_sol < self.sol_max):
            t_call = sc.stats.expon.rvs(loc=0,scale=self.media_call)
            yield self.env.timeout(t_call)
            self.cant_sol += 1
            self.tiempo_espera_tot -= self.env.now
            u = np.random.rand()
            if (u < self.proba_b1 and (self.cant_sol < self.sol_max)):
                self.env.process(self.usar_bdd(1))
            elif (u < self.proba_b1 and (self.cant_sol == self.sol_max)):
                yield self.env.process(self.usar_bdd(1))
            elif (u >= self.proba_b1 and (self.cant_sol < self.sol_max)):
                self.env.process(self.usar_bdd(2))
            else:
                yield self.env.process(self.usar_bdd(2))
                    
        self.tiempo_cola += self.tiempo_espera_tot
        print("Transcurrieron", self.env.now, "segundos en", self.sol_max, "iteraciones")         
        print("Se uso", self.usos_bdd[0], "veces la bdd1 y",  self.usos_bdd[1], "veces la bdd2")
        print("Hubo", self.cant_sol_sin_espera, "llamadas sin espera, es decir", self.cant_sol_sin_espera/self.sol_max, "del total")
        print("El tiempo medio de espera fue:", self.tiempo_espera_tot/self.sol_max, "seg")
        print("El tiempo medio de cola (sin procesamiento de llamada) fue:", self.tiempo_cola/self.sol_max, "seg")
        
    def usar_bdd(self, n_bdd):
        if (n_bdd == 1):
            self.usos_bdd[0] += 1
            if (self.bdd_1.count == 0):
                self.cant_sol_sin_espera += 1
            with self.bdd_1.request() as bdd1:
                yield bdd1
                t_uso = sc.stats.expon.rvs(loc=0, scale=self.media_1)
                yield self.env.timeout(t_uso)
        else:
            self.usos_bdd[1] += 1
            if (self.bdd_2.count == 0):
                self.cant_sol_sin_espera += 1
            with self.bdd_2.request() as bdd2:
                yield bdd2
                t_uso = sc.stats.expon.rvs(loc=0, scale=self.media_2)
                yield self.env.timeout(t_uso)
        self.tiempo_espera_tot += self.env.now
        self.tiempo_cola -= t_uso
                

env = simpy.Environment()
base1 = simpy.Resource(env, capacity=1)
base2 = simpy.Resource(env, capacity=1)
ws1 = web_service_1(env, 4, base1, base2, 0.6, 0.7, 1, 100000)
env.run()

Transcurrieron 399754.0866933777 segundos en 100000 iteraciones
Se uso 59983 veces la bdd1 y 40017 veces la bdd2
Hubo 89710 llamadas sin espera, es decir 0.8971 del total
El tiempo medio de espera fue: 0.9115069553857076 seg
El tiempo medio de cola (sin procesamiento de llamada) fue: 0.09262824954041236 seg


In [37]:
class web_service_2:
    def __init__(self, env, media_call, bdd, media_bdd, sol_max):
        self.env = env
        self.media_call = media_call
        self.bdd = bdd
        self.media_bdd = media_bdd
        self.sol_max = sol_max
        self.action = env.process(self.run(env))
        self.cant_sol = 0
        self.cant_sol_sin_espera = 0
        self.tiempo_cola = 0
        self.tiempo_espera_tot = 0 #Sumatoria Tiempo_proc_i - Tiempo_llamada_i =
                                   # (Sumatoria Tiempo_proc_i) - (Sumatoria Tiempo_llamada_k)
    
    
    def run(self, env):
        while(self.cant_sol < self.sol_max):
            yield self.env.timeout(sc.stats.expon.rvs(loc=0,scale=self.media_call))
            self.tiempo_espera_tot -= self.env.now
            self.cant_sol += 1
            if (self.cant_sol < self.sol_max):
                self.env.process(self.usar_bdd())
            else:
                yield self.env.process(self.usar_bdd())
                    
        self.tiempo_cola += self.tiempo_espera_tot
        print("Transcurrieron", self.env.now, "segundos en", self.sol_max, "iteraciones")
        print("Hubo", self.cant_sol_sin_espera, "llamadas sin espera, es decir", self.cant_sol_sin_espera/self.sol_max, "del total")
        print("El tiempo medio de espera fue:", self.tiempo_espera_tot/self.sol_max, "seg")
        print("El tiempo medio de cola (sin procesamiento de llamada) fue:", self.tiempo_cola/self.sol_max, "seg")
    
    def usar_bdd(self):
        if (self.bdd.count == 0):
            self.cant_sol_sin_espera += 1
        with self.bdd.request() as bdd:
            yield bdd
            t_uso = sc.stats.expon.rvs(loc=0, scale=self.media_bdd)
            yield self.env.timeout(t_uso)
        self.tiempo_espera_tot += self.env.now
        self.tiempo_cola -= t_uso

env = simpy.Environment()
base = simpy.Resource(env, capacity=1)
ws2 = web_service_2(env, 4, base, 0.8, 100000)
env.run()

Transcurrieron 400075.0848755968 segundos en 100000 iteraciones
Hubo 80140 llamadas sin espera, es decir 0.8014 del total
El tiempo medio de espera fue: 0.9984850016983448 seg
El tiempo medio de cola (sin procesamiento de llamada) fue: 0.19990797502652494 seg


## Debido a que el tiempo medio de espera solo 8% mas rápido en la opcion 1 (0.91 segundos contra los 0.99 de la opcion 2), entonces dado que la opcion 2 es mas económica recomendamos elegir esa.